In [1]:
import lusid
import lusid.models as models
import lusid.api as la
import lusid.models as lm
from lusid import ApiException
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.seed_sample_data import seed_data
from lusidtools.cocoon.utilities import create_scope_id
from lusidtools.cocoon.cocoon_printer import (
    format_instruments_response,
    format_portfolios_response,
    format_transactions_response,
    format_quotes_response,
    format_holdings_response,
)

from collections import defaultdict
import pandas as pd
import numpy as np
import json
import openpyxl
import inspect
import os

m = inspect.getmembers(lusid.utilities) # get module content
print(list(filter(lambda x: inspect.ismodule(x[1]), m))) # filter dependant modules

pd.set_option('display.max_columns', None)

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")

print ('LUSID Environment Initialised')
print ('API Version: ', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

[('api_client_builder', <module 'lusid.utilities.api_client_builder' from '/c/code/lusid/sample-notebooks/venv-0.0.1/lib/python3.7/site-packages/lusid/utilities/api_client_builder.py'>), ('api_client_factory', <module 'lusid.utilities.api_client_factory' from '/c/code/lusid/sample-notebooks/venv-0.0.1/lib/python3.7/site-packages/lusid/utilities/api_client_factory.py'>), ('api_configuration', <module 'lusid.utilities.api_configuration' from '/c/code/lusid/sample-notebooks/venv-0.0.1/lib/python3.7/site-packages/lusid/utilities/api_configuration.py'>), ('api_configuration_loader', <module 'lusid.utilities.api_configuration_loader' from '/c/code/lusid/sample-notebooks/venv-0.0.1/lib/python3.7/site-packages/lusid/utilities/api_configuration_loader.py'>), ('lusid_retry', <module 'lusid.utilities.lusid_retry' from '/c/code/lusid/sample-notebooks/venv-0.0.1/lib/python3.7/site-packages/lusid/utilities/lusid_retry.py'>), ('proxy_config', <module 'lusid.utilities.proxy_config' from '/c/code/lusid

In [2]:
orders_scope = "orders"
orders_portfolio = "us_long_live"
ibor_scope = "middle_office"
ibor_portfolio = "us_long"
strategy_shk = "strategy"
executed_price_code = "executed_price"
execution_price_property = f"Allocation/{orders_scope}/{executed_price_code}"
order_strategy_property = f"Transaction/{orders_scope}/{strategy_shk}"
allocation_strategy_property = f"Allocation/{orders_scope}/{strategy_shk}"
pf_created_date = "2020-01-01T00:00:00+00:00"

In [3]:
instr_df = pd.read_csv("data/multiplecurrencies-instruments-2.csv")
display(instr_df)

,instrument_name,client_internal,currency,isin,figi,exchange_code,country_issue,ticker,market_sector,security_type,coupon
0,IBM,imd_34634534,USD,US0231351067,BBG000BVPXP1,UN,united_states_america,IBM,equity,common_stock,NaN


In [4]:
instrument_mapping = {
    "identifier_mapping": {
        "ClientInternal": "client_internal",
        "Isin": "isin",
        "Figi": "figi",
    },
    "required": {
        "name": "instrument_name"
    },
}

In [5]:
result = load_from_data_frame(
    api_factory=api_factory,
    scope=orders_scope,
    data_frame=instr_df,
    mapping_required=instrument_mapping["required"],
    mapping_optional={},
    file_type="instruments",
    identifier_mapping=instrument_mapping["identifier_mapping"],
)

succ, failed, errors = format_instruments_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}])

,success,failed,errors
0,1,0,0


In [6]:
properties_api = api_factory.build(la.PropertyDefinitionsApi)

def create_property_definition(domain, scope, code, data_type):
    try:
        properties_api.create_property_definition(
            lm.CreatePropertyDefinitionRequest(
                domain=domain,
                scope=scope,
                code=code,
                display_name=code,
                life_time="Perpetual",
                value_required=False,
                data_type_id=lm.resource_id.ResourceId(scope="system", code=data_type)
            )
        )
    except ApiException as e:
        detail = json.loads(e.body)
        if detail["code"] != 124:  # 'PropertyAlreadyExists'
            raise e

In [7]:
# sub-holding keys
create_property_definition("Transaction", orders_scope, strategy_shk, "string")

# allocation execution price
create_property_definition("Allocation", orders_scope, executed_price_code, "number")

In [8]:
def create_portfolio(scope, portfolio_code, name):

    pf_df = pd.DataFrame(data=[
        {"portfolio_code": portfolio_code, "portfolio_name": name},
    ])
    
    portfolio_mapping = {
        "required": {
            "code": "portfolio_code",
            "display_name": "portfolio_name",
            "base_currency": "$USD",
        },
        "optional": {
            "created": f"${pf_created_date}"
        },
    }
    
    result = load_from_data_frame(
        api_factory=api_factory,
        scope=scope,
        data_frame=pf_df,
        mapping_required=portfolio_mapping["required"],
        mapping_optional=portfolio_mapping["optional"],
        file_type="portfolios",
        sub_holding_keys=[strategy_shk],
    )

    succ, failed = format_portfolios_response(result)
    display(pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}])) 
    print(list(failed['error_items']))

In [9]:
create_portfolio(orders_scope, orders_portfolio, "ibor")

,success,failed,errors
0,1,0,0


[]


### Set holdings in portfolio

In [10]:
hldgs_df = pd.read_csv("data/initial_holdings-2.csv")
display(hldgs_df)

,instrument_name,ccyId,client_internal,isin,figi,quantity,unit_cost,total_cost,currency,strategy
0,IBM,NaN,imd_34634534,US0231351067,BBG000BVPXP1,1000,150,150000,USD,Tech
1,IBM,NaN,imd_34634534,US0231351067,BBG000BVPXP1,200,150,30000,USD,Balanced
2,USD,USD,NaN,NaN,NaN,1000,1,1000,USD,NaN


In [11]:
holdings_mapping = {
    "required":{
        "code": f"${orders_portfolio}",
        "effective_at": "$2020-05-01",
        "tax_lots.units": "quantity"
    },
    "identifier_mapping": {
        "Figi": "figi",
        "Currency": "ccyId"
    },
    "optional": {
        "tax_lots.cost.amount": "total_cost",
        "tax_lots.cost.currency": "currency",
        "tax_lots.price": "unit_cost"
    }
}

In [12]:
result = load_from_data_frame(
    api_factory=api_factory,
    scope=orders_scope,
    data_frame=hldgs_df,
    mapping_required=holdings_mapping["required"],
    mapping_optional=holdings_mapping["optional"],
    identifier_mapping=holdings_mapping["identifier_mapping"],
    sub_holding_keys=[strategy_shk],
    file_type="holdings"
)

succ, failed = format_holdings_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}])

,success,failed,errors
0,1,0,0


In [13]:
executed_holdings = api_factory.build(la.TransactionPortfoliosApi).get_holdings(scope=orders_scope, code=orders_portfolio, property_keys=["Instrument/default/Name"])

lusid_response_to_data_frame(executed_holdings, rename_properties=True)

,instrument_uid,strategy(orders-SubHoldingKeys),Name(default-Properties),SourcePortfolioId(default-Properties),SourcePortfolioScope(default-Properties),holding_type,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount,cost_portfolio_ccy.currency
0,LUID_HTCHLW5O,Tech,IBM,us_long_live,orders,P,1000.0,1000.0,150000.0,USD,150000.0,USD
1,LUID_HTCHLW5O,Balanced,IBM,us_long_live,orders,P,200.0,200.0,30000.0,USD,30000.0,USD
2,CCY_USD,<Not Classified>,CCY_USD,us_long_live,orders,B,1000.0,1000.0,1000.0,USD,1000.0,USD


### Publish some orders

In [14]:
orders_df = pd.read_csv('data/orders-2.csv')
orders_df

,portfolio,instrument_name,client_internal,isin,figi,quantity,price,currency,order_id,side,strategy
0,us_long_live,IBM,imd_34634534,US0231351067,BBG000BVPXP1,100,0.9,USD,ORD001,buy limit,Tech
1,us_long_live,IBM,imd_34634534,US0231351067,BBG000BVPXP1,75,1.1,USD,ORD002,sell limit,Tech


In [15]:
order_requests = defaultdict(list)
order_sets = defaultdict(list)

for index, order in orders_df.iterrows():
    
    portfolio = order['portfolio']

    order_requests[portfolio].append(
        models.OrderRequest(
            id=models.ResourceId(
                scope=orders_scope,
                code=order['order_id']
            ),
            quantity=order['quantity'],
            side=order['side'],
            instrument_identifiers={
                'Instrument/default/Figi': order['figi'],
                'Instrument/default/Name': order['instrument_name']
            },
            properties={
                "Order/system/Currency":
                    models.PerpetualProperty(key="Order/system/Currency", 
                                             value=models.PropertyValue(order['currency'])),
                "Order/system/LimitPrice":
                    models.PerpetualProperty(key="Order/system/LimitPrice", 
                                             value=models.PropertyValue(order['price'])),
                "Order/system/State":
                    models.PerpetualProperty(key="Order/system/State", 
                                             value=models.PropertyValue("Open")),
                f"Transaction/{orders_scope}/{strategy_shk}":
                    models.PerpetualProperty(key=f"Transaction/{orders_scope}/{strategy_shk}", 
                                             value=models.PropertyValue(order['strategy']))
            },
            order_book_id=models.ResourceId(
                scope=orders_scope,
                code=orders_portfolio,
            ),
            portfolio_id=models.ResourceId(
                scope=orders_scope,
                code=order['portfolio']
            )
        )
    )

for order_portfolio in order_requests:
    order_sets[order_portfolio] = models.OrderSetRequest(
        order_requests=order_requests[order_portfolio]
    )    

for order_portfolio in order_sets:    
    response = api_factory.build(lusid.api.OrdersApi).upsert_orders(
        request=order_sets[order_portfolio]
    )
    
    display(lusid_response_to_data_frame(response, rename_properties=True))

,Currency(system-Properties),LimitPrice(system-Properties),State(system-Properties),strategy(orders-Properties),version.effective_from,version.as_at_date,instrument_identifiers.Instrument/default/Figi,instrument_identifiers.Instrument/default/Name,quantity,side,order_book_id.scope,order_book_id.code,portfolio_id.scope,portfolio_id.code,id.scope,id.code,lusid_instrument_id
0,USD,1.1,Open,Tech,0001-01-01 00:00:00+00:00,2020-06-15 13:26:15.961373+00:00,BBG000BVPXP1,IBM,75,sell limit,orders,us_long_live,orders,us_long_live,orders,ORD002,LUID_HTCHLW5O
1,USD,0.9,Open,Tech,0001-01-01 00:00:00+00:00,2020-06-15 13:26:15.961373+00:00,BBG000BVPXP1,IBM,100,buy limit,orders,us_long_live,orders,us_long_live,orders,ORD001,LUID_HTCHLW5O


### ... and show holdings with orders

In [16]:
executed_holdings = api_factory.build(la.TransactionPortfoliosApi).get_holdings_with_orders(scope=orders_scope, code=orders_portfolio, property_keys=["Instrument/default/Name"])

lusid_response_to_data_frame(executed_holdings, rename_properties=True)

,instrument_uid,strategy(orders-SubHoldingKeys),Name(default-Properties),SourcePortfolioId(default-Properties),SourcePortfolioScope(default-Properties),holding_type,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount,cost_portfolio_ccy.currency,transaction.transaction_id,transaction.type,transaction.instrument_identifiers.Instrument/default/Figi,transaction.instrument_identifiers.Instrument/default/Name,transaction.instrument_uid,transaction.transaction_date,transaction.settlement_date,transaction.units,transaction.transaction_price.price,transaction.transaction_price.type,transaction.total_consideration.amount,transaction.total_consideration.currency,transaction.exchange_rate,transaction.transaction_currency,transaction.properties.Transaction/default/TradeToPortfolioRate.key,transaction.properties.Transaction/default/TradeToPortfolioRate.value.metric_value.value,transaction.properties.Transaction/default/TradeToPortfolioRate.value.metric_value.unit,transaction.source
0,LUID_HTCHLW5O,Tech,IBM,us_long_live,orders,P,1000.0,1000.0,150000.0,USD,150000.0,USD,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LUID_HTCHLW5O,Balanced,IBM,us_long_live,orders,P,200.0,200.0,30000.0,USD,30000.0,USD,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,LUID_HTCHLW5O,Tech,IBM,us_long_live,orders,O,-75.0,0.0,82.5,USD,82.5,USD,VirtualTrade-Order-orders/ORD002-a508f,Sell,BBG000BVPXP1,IBM,LUID_HTCHLW5O,2020-06-15 13:26:16.138299+00:00,2020-06-18 13:26:16.138299+00:00,-75.0,1.1,Price,82.5,USD,1.0,USD,Transaction/default/TradeToPortfolioRate,1.0,,
3,LUID_HTCHLW5O,Tech,IBM,us_long_live,orders,O,100.0,0.0,-90.0,USD,-90.0,USD,VirtualTrade-Order-orders/ORD001-1c6f3,Buy,BBG000BVPXP1,IBM,LUID_HTCHLW5O,2020-06-15 13:26:16.138299+00:00,2020-06-18 13:26:16.138299+00:00,100.0,0.9,Price,-90.0,USD,1.0,USD,Transaction/default/TradeToPortfolioRate,1.0,,
4,CCY_USD,<Not Classified>,CCY_USD,us_long_live,orders,B,1000.0,1000.0,1000.0,USD,1000.0,USD,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,CCY_USD,Tech,CCY_USD,us_long_live,orders,OC,82.5,0.0,-82.5,USD,-82.5,USD,VirtualTrade-Order-orders/ORD002-a508f,Sell,BBG000BVPXP1,IBM,LUID_HTCHLW5O,2020-06-15 13:26:16.138299+00:00,2020-06-18 13:26:16.138299+00:00,-75.0,1.1,Price,82.5,USD,1.0,USD,Transaction/default/TradeToPortfolioRate,1.0,,
6,CCY_USD,Tech,CCY_USD,us_long_live,orders,OC,-90.0,0.0,90.0,USD,90.0,USD,VirtualTrade-Order-orders/ORD001-1c6f3,Buy,BBG000BVPXP1,IBM,LUID_HTCHLW5O,2020-06-15 13:26:16.138299+00:00,2020-06-18 13:26:16.138299+00:00,100.0,0.9,Price,-90.0,USD,1.0,USD,Transaction/default/TradeToPortfolioRate,1.0,,


### ... and publish some allocations

In [17]:
allocations_df = pd.read_csv('data/allocations-2.csv')
allocations_df

,portfolio,instrument_name,client_internal,isin,figi,quantity,price,currency,allocation_id,originating_order,strategy,side
0,us_long_live,IBM,imd_34634534,US0231351067,BBG000BVPXP1,82,0.9,USD,ALLOC001,ORD001,Tech,buy limit
1,us_long_live,IBM,imd_34634534,US0231351067,BBG000BVPXP1,61,1.1,USD,ALLOC002,ORD002,Tech,sell limit


In [18]:
allocation_requests = defaultdict(list)
allocation_sets = defaultdict(list)

for index, allocation in allocations_df.iterrows():
    
    portfolio = allocation['portfolio']

    allocation_requests[portfolio].append(
        models.AllocationRequest(
            id=models.ResourceId(
                scope=orders_scope,
                code=allocation['allocation_id']
            ),
            allocated_order_id=models.ResourceId(
                scope=orders_scope,
                code=allocation['originating_order']
            ),
            quantity=allocation['quantity'],
            instrument_identifiers={
                'Instrument/default/Figi': allocation['figi'],
                'Instrument/default/Name': allocation['instrument_name']
            },
            properties={
                "Allocation/system/Currency":
                    models.PerpetualProperty(key="Allocation/system/Currency", 
                                             value=models.PropertyValue(allocation['currency'])),
                "Allocation/system/LimitPrice":
                    models.PerpetualProperty(key="Allocation/system/LimitPrice", 
                                             value=models.PropertyValue(allocation['price'])),
                "Allocation/system/Side":
                    models.PerpetualProperty(key="Allocation/system/Side", 
                                             value=models.PropertyValue(allocation['side'])),
                "Allocation/system/State":
                    models.PerpetualProperty(key="Allocation/system/State", 
                                             value=models.PropertyValue("Open")),
                f"Transaction/{orders_scope}/{strategy_shk}":
                    models.PerpetualProperty(key=f"Transaction/{orders_scope}/{strategy_shk}", 
                                             value=models.PropertyValue(allocation['strategy']))
            },
            portfolio_id=models.ResourceId(
                scope=orders_scope,
                code=allocation['portfolio']
            )
        )
    )

for allocation_portfolio in allocation_requests:
    allocation_sets[allocation_portfolio] = models.AllocationSetRequest(
        allocation_requests=allocation_requests[allocation_portfolio]
    )    

for allocation_portfolio in allocation_sets:    
    response = api_factory.build(lusid.api.AllocationsApi).upsert_allocations(
        request=allocation_sets[allocation_portfolio]
    )
    
    display(lusid_response_to_data_frame(response, rename_properties=True))

,id.scope,id.code,allocated_order_id.scope,allocated_order_id.code,portfolio_id.scope,portfolio_id.code,quantity,instrument_identifiers.Instrument/default/Figi,instrument_identifiers.Instrument/default/Name,version.effective_from,version.as_at_date,Currency(system-Properties),LimitPrice(system-Properties),Side(system-Properties),State(system-Properties),strategy(orders-Properties),lusid_instrument_id
0,orders,ALLOC001,orders,ORD001,orders,us_long_live,82,BBG000BVPXP1,IBM,0001-01-01 00:00:00+00:00,2020-06-15 13:26:27.839998+00:00,USD,0.9,buy limit,Open,Tech,LUID_HTCHLW5O
1,orders,ALLOC002,orders,ORD002,orders,us_long_live,61,BBG000BVPXP1,IBM,0001-01-01 00:00:00+00:00,2020-06-15 13:26:27.839998+00:00,USD,1.1,sell limit,Open,Tech,LUID_HTCHLW5O


### ... and look at holdings again

In [19]:
executed_holdings = api_factory.build(la.TransactionPortfoliosApi).get_holdings_with_orders(scope=orders_scope, code=orders_portfolio, property_keys=["Instrument/default/Name"])

lusid_response_to_data_frame(executed_holdings, rename_properties=True)

,instrument_uid,strategy(orders-SubHoldingKeys),Name(default-Properties),SourcePortfolioId(default-Properties),SourcePortfolioScope(default-Properties),holding_type,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount,cost_portfolio_ccy.currency,transaction.transaction_id,transaction.type,transaction.instrument_identifiers.Instrument/default/Figi,transaction.instrument_identifiers.Instrument/default/Name,transaction.instrument_uid,transaction.transaction_date,transaction.settlement_date,transaction.units,transaction.transaction_price.price,transaction.transaction_price.type,transaction.total_consideration.amount,transaction.total_consideration.currency,transaction.exchange_rate,transaction.transaction_currency,transaction.properties.Transaction/default/TradeToPortfolioRate.key,transaction.properties.Transaction/default/TradeToPortfolioRate.value.metric_value.value,transaction.properties.Transaction/default/TradeToPortfolioRate.value.metric_value.unit,transaction.source
0,LUID_HTCHLW5O,Tech,IBM,us_long_live,orders,P,1000.0,1000.0,150000.0,USD,150000.0,USD,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LUID_HTCHLW5O,Balanced,IBM,us_long_live,orders,P,200.0,200.0,30000.0,USD,30000.0,USD,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,LUID_HTCHLW5O,Tech,IBM,us_long_live,orders,O,-14.0,0.0,15.4,USD,15.4,USD,VirtualTrade-Order-orders/ORD002-ce2f9,Sell,BBG000BVPXP1,IBM,LUID_HTCHLW5O,2020-06-15 13:26:28.019519+00:00,2020-06-18 13:26:28.019519+00:00,-14.0,1.1,Price,15.4,USD,1.0,USD,Transaction/default/TradeToPortfolioRate,1.0,,
3,LUID_HTCHLW5O,Tech,IBM,us_long_live,orders,O,18.0,0.0,-16.2,USD,-16.2,USD,VirtualTrade-Order-orders/ORD001-9497c,Buy,BBG000BVPXP1,IBM,LUID_HTCHLW5O,2020-06-15 13:26:28.019519+00:00,2020-06-18 13:26:28.019519+00:00,18.0,0.9,Price,-16.2,USD,1.0,USD,Transaction/default/TradeToPortfolioRate,1.0,,
4,LUID_HTCHLW5O,Tech,IBM,us_long_live,orders,L,-61.0,0.0,67.1,USD,67.1,USD,VirtualTrade-Order-orders/ALLOC002-a363b,Sell,BBG000BVPXP1,IBM,LUID_HTCHLW5O,2020-06-15 13:26:28.019519+00:00,2020-06-18 13:26:28.019519+00:00,-61.0,1.1,Price,67.1,USD,1.0,USD,Transaction/default/TradeToPortfolioRate,1.0,,
5,LUID_HTCHLW5O,Tech,IBM,us_long_live,orders,L,82.0,0.0,-73.8,USD,-73.8,USD,VirtualTrade-Order-orders/ALLOC001-4784b,Buy,BBG000BVPXP1,IBM,LUID_HTCHLW5O,2020-06-15 13:26:28.019519+00:00,2020-06-18 13:26:28.019519+00:00,82.0,0.9,Price,-73.8,USD,1.0,USD,Transaction/default/TradeToPortfolioRate,1.0,,
6,CCY_USD,<Not Classified>,CCY_USD,us_long_live,orders,B,1000.0,1000.0,1000.0,USD,1000.0,USD,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,CCY_USD,Tech,CCY_USD,us_long_live,orders,OC,15.4,0.0,-15.4,USD,-15.4,USD,VirtualTrade-Order-orders/ORD002-ce2f9,Sell,BBG000BVPXP1,IBM,LUID_HTCHLW5O,2020-06-15 13:26:28.019519+00:00,2020-06-18 13:26:28.019519+00:00,-14.0,1.1,Price,15.4,USD,1.0,USD,Transaction/default/TradeToPortfolioRate,1.0,,
8,CCY_USD,Tech,CCY_USD,us_long_live,orders,OC,-16.2,0.0,16.2,USD,16.2,USD,VirtualTrade-Order-orders/ORD001-9497c,Buy,BBG000BVPXP1,IBM,LUID_HTCHLW5O,2020-06-15 13:26:28.019519+00:00,2020-06-18 13:26:28.019519+00:00,18.0,0.9,Price,-16.2,USD,1.0,USD,Transaction/default/TradeToPortfolioRate,1.0,,
9,CCY_USD,Tech,CCY_USD,us_long_live,orders,LC,67.1,0.0,-67.1,USD,-67.1,USD,VirtualTrade-Order-orders/ALLOC002-a363b,Sell,BBG000BVPXP1,IBM,LUID_HTCHLW5O,2020-06-15 13:26:28.019519+00:00,2020-06-18 13:26:28.019519+00:00,-61.0,1.1,Price,67.1,USD,1.0,USD,Transaction/default/TradeToPortfolioRate,1.0,,


### Now close the orders and allocations

In [23]:
order_requests = defaultdict(list)
order_sets = defaultdict(list)

for index, order in orders_df.iterrows():
    
    portfolio = order['portfolio']

    order_requests[portfolio].append(
        models.OrderRequest(
            id=models.ResourceId(
                scope=orders_scope,
                code=order['order_id']
            ),
            quantity=order['quantity'],
            side=order['side'],
            instrument_identifiers={
                'Instrument/default/Figi': order['figi'],
                'Instrument/default/Name': order['instrument_name']
            },
            properties={
                "Order/system/Currency":
                    models.PerpetualProperty(key="Order/system/Currency", 
                                             value=models.PropertyValue(order['currency'])),
                "Order/system/LimitPrice":
                    models.PerpetualProperty(key="Order/system/LimitPrice", 
                                             value=models.PropertyValue(order['price'])),
                "Order/system/State":
                    models.PerpetualProperty(key="Order/system/State", 
                                             value=models.PropertyValue("Closed")),
                f"Transaction/{orders_scope}/{strategy_shk}":
                    models.PerpetualProperty(key=f"Transaction/{orders_scope}/{strategy_shk}", 
                                             value=models.PropertyValue(order['strategy']))
            },
            order_book_id=models.ResourceId(
                scope=orders_scope,
                code=orders_portfolio,
            ),
            portfolio_id=models.ResourceId(
                scope=orders_scope,
                code=order['portfolio']
            )
        )
    )

for order_portfolio in order_requests:
    order_sets[order_portfolio] = models.OrderSetRequest(
        order_requests=order_requests[order_portfolio]
    )    

for order_portfolio in order_sets:    
    response = api_factory.build(lusid.api.OrdersApi).upsert_orders(
        request=order_sets[order_portfolio]
    )
    
    display(lusid_response_to_data_frame(response, rename_properties=True))
    
allocation_requests = defaultdict(list)
allocation_sets = defaultdict(list)

for index, allocation in allocations_df.iterrows():
    
    portfolio = allocation['portfolio']

    allocation_requests[portfolio].append(
        models.AllocationRequest(
            id=models.ResourceId(
                scope=orders_scope,
                code=allocation['allocation_id']
            ),
            allocated_order_id=models.ResourceId(
                scope=orders_scope,
                code=allocation['originating_order']
            ),
            quantity=allocation['quantity'],
            instrument_identifiers={
                'Instrument/default/Figi': allocation['figi'],
                'Instrument/default/Name': allocation['instrument_name']
            },
            properties={
                "Allocation/system/Currency":
                    models.PerpetualProperty(key="Allocation/system/Currency", 
                                             value=models.PropertyValue(allocation['currency'])),
                "Allocation/system/LimitPrice":
                    models.PerpetualProperty(key="Allocation/system/LimitPrice", 
                                             value=models.PropertyValue(allocation['price'])),
                "Allocation/system/Side":
                    models.PerpetualProperty(key="Allocation/system/Side", 
                                             value=models.PropertyValue(allocation['side'])),
                "Allocation/system/State":
                    models.PerpetualProperty(key="Allocation/system/State", 
                                             value=models.PropertyValue("Closed")),
                f"Transaction/{orders_scope}/{strategy_shk}":
                    models.PerpetualProperty(key=f"Transaction/{orders_scope}/{strategy_shk}", 
                                             value=models.PropertyValue(allocation['strategy']))
            },
            portfolio_id=models.ResourceId(
                scope=orders_scope,
                code=allocation['portfolio']
            )
        )
    )

for allocation_portfolio in allocation_requests:
    allocation_sets[allocation_portfolio] = models.AllocationSetRequest(
        allocation_requests=allocation_requests[allocation_portfolio]
    )    

for allocation_portfolio in allocation_sets:    
    response = api_factory.build(lusid.api.AllocationsApi).upsert_allocations(
        request=allocation_sets[allocation_portfolio]
    )
    
    display(lusid_response_to_data_frame(response, rename_properties=True))

,Currency(system-Properties),LimitPrice(system-Properties),State(system-Properties),strategy(orders-Properties),version.effective_from,version.as_at_date,instrument_identifiers.Instrument/default/Figi,instrument_identifiers.Instrument/default/Name,quantity,side,order_book_id.scope,order_book_id.code,portfolio_id.scope,portfolio_id.code,id.scope,id.code,lusid_instrument_id
0,USD,1.1,Closed,Tech,0001-01-01 00:00:00+00:00,2020-06-15 13:29:17.096570+00:00,BBG000BVPXP1,IBM,75,sell limit,orders,us_long_live,orders,us_long_live,orders,ORD002,LUID_HTCHLW5O
1,USD,0.9,Closed,Tech,0001-01-01 00:00:00+00:00,2020-06-15 13:29:17.096570+00:00,BBG000BVPXP1,IBM,100,buy limit,orders,us_long_live,orders,us_long_live,orders,ORD001,LUID_HTCHLW5O


,id.scope,id.code,allocated_order_id.scope,allocated_order_id.code,portfolio_id.scope,portfolio_id.code,quantity,instrument_identifiers.Instrument/default/Figi,instrument_identifiers.Instrument/default/Name,version.effective_from,version.as_at_date,Currency(system-Properties),LimitPrice(system-Properties),Side(system-Properties),State(system-Properties),strategy(orders-Properties),lusid_instrument_id
0,orders,ALLOC001,orders,ORD001,orders,us_long_live,82,BBG000BVPXP1,IBM,0001-01-01 00:00:00+00:00,2020-06-15 13:29:17.152649+00:00,USD,0.9,buy limit,Closed,Tech,LUID_HTCHLW5O
1,orders,ALLOC002,orders,ORD002,orders,us_long_live,61,BBG000BVPXP1,IBM,0001-01-01 00:00:00+00:00,2020-06-15 13:29:17.152649+00:00,USD,1.1,sell limit,Closed,Tech,LUID_HTCHLW5O


### And look at holdings for the final time

In [24]:
executed_holdings = api_factory.build(la.TransactionPortfoliosApi).get_holdings_with_orders(scope=orders_scope, code=orders_portfolio, property_keys=["Instrument/default/Name"])

lusid_response_to_data_frame(executed_holdings, rename_properties=True)

,instrument_uid,strategy(orders-SubHoldingKeys),Name(default-Properties),SourcePortfolioId(default-Properties),SourcePortfolioScope(default-Properties),holding_type,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount,cost_portfolio_ccy.currency
0,LUID_HTCHLW5O,Tech,IBM,us_long_live,orders,P,1000.0,1000.0,150000.0,USD,150000.0,USD
1,LUID_HTCHLW5O,Balanced,IBM,us_long_live,orders,P,200.0,200.0,30000.0,USD,30000.0,USD
2,CCY_USD,<Not Classified>,CCY_USD,us_long_live,orders,B,1000.0,1000.0,1000.0,USD,1000.0,USD
